In [9]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


# 测试整体RAG查询流程

In [10]:
# 数据加载

import pandas as pd
from src.utils.embedding_utils import load_df_from_csv_with_embedding
from src.utils.config import OUTPUT_PATH

entities_with_embeddings = load_df_from_csv_with_embedding("/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/export/multihop_rag/final_entities_with_embeddings_by_chatgpt4o.csv", ['entity_embedding'])
entities_with_clusters = pd.read_csv("/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/export/multihop_rag/entities_cluster_result_Oct24th.csv")
relationships = pd.read_csv("/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/export/multihop_rag/relationships_with_SAMENAME_Oct24th.csv")

# S1：选项包括非保护组相关项以及保护组不相关项

In [3]:
from src.utils.embedding_utils import find_k_nearest_entities


# 获取查询嵌入
query = "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"

# 查询最相近的 k 个实体
k = 1
nearest_entities = find_k_nearest_entities(query, entities_with_embeddings, k=k)
nearest_entities

,entity_name,entity_type,description,entity_id,corpus_id,entity_embedding,similarity
13127,BANKMAN-FRIED,PERSON,Bankman-Fried is a key figure in the cryptocur...,13127,1452,"[0.01225276943296194, -0.012308556586503983, -...",0.857498


In [4]:
from src.report.community_report import generate_seperated_reports

entity_id=nearest_entities["entity_id"].iloc[0]

seperated_reports=generate_seperated_reports(entity_id, entities_with_clusters, relationships, output_file=f'export/rag_test/seperated_community_report_{entity_id}.json')

Merged reports saved to export/rag_test/seperated_community_report_13127.json


In [5]:
from src.utils.message_templates import generate_multiple_choice_query_template
from src.utils.llm_helpers import call_llm_api
import json

# 遍历每一份报告
for report in seperated_reports:
    cluster_column = report["cluster_column"]  # 聚类列名称
    report_content = report["report"]         # 报告内容
    
    multiple_choice = {
        "A": "Sam Bankman-Fried",
        "B": "Donald Trump",
        "C": "Taylor Swift",
        "D": "Unknown / Insufficent Information"
    }
    query_template=generate_multiple_choice_query_template(query,multiple_choice,report_content)
    answer=call_llm_api(query_template)
    print(answer)

```json
{
    "answer": "A: Sam Bankman-Fried",
    "used_findings": [1, 4, 5]
}
```
```json
{
    "answer": "A: Sam Bankman-Fried",
    "used_findings": [1, 4]
}
```
```json
{
    "answer": "A: Sam Bankman-Fried",
    "used_findings": [1, 2, 3]
}
```
```json
{
    "answer": "A: Sam Bankman-Fried",
    "used_findings": [1, 2, 5, 6, 8]
}
```
```json
{
    "answer": "A: Sam Bankman-Fried",
    "used_findings": [1, 4, 5]
}
```


# S2：选项包括：保护组相关项以及非保护组不相关项

In [6]:
from src.utils.embedding_utils import find_k_nearest_entities


# 获取查询嵌入
query = "Which pop star, recognized by Time as Person of the Year, was seen supporting a football team at Arrowhead Stadium where she was personally invited by a player who also made her a friendship bracelet with his number on it, as reported by 'The Age', 'The Independent - Life and Style', and 'CBSSports.com'?"

# 查询最相近的 k 个实体
k = 1
nearest_entities = find_k_nearest_entities(query, entities_with_embeddings, k=k)
nearest_entities

,entity_name,entity_type,description,entity_id,corpus_id,entity_embedding,similarity
1185,ARROWHEAD STADIUM,LOCATION,Arrowhead Stadium is a sports venue where Trav...,1185,132,"[0.030295245349407196, -0.0038188539911061525,...",0.845767


In [7]:
from src.report.community_report import generate_seperated_reports

entity_id=nearest_entities["entity_id"].iloc[0]

seperated_reports=generate_seperated_reports(entity_id, entities_with_clusters, relationships, output_file=f'export/rag_test/seperated_community_report_{entity_id}.json')

Merged reports saved to export/rag_test/seperated_community_report_1185.json


In [8]:
from src.utils.message_templates import generate_multiple_choice_query_template
from src.utils.llm_helpers import call_llm_api
import json

# 遍历每一份报告
for report in seperated_reports:
    cluster_column = report["cluster_column"]  # 聚类列名称
    report_content = report["report"]         # 报告内容
    
    multiple_choice = {
        "A": "Sam Bankman-Fried",
        "B": "Donald Trump",
        "C": "Taylor Swift",
        "D": "Unknown / Insufficent Information"
    }
    query_template=generate_multiple_choice_query_template(query,multiple_choice,report_content)
    answer=call_llm_api(query_template)
    print(answer)

```json
{
    "answer": "C: Taylor Swift",
    "used_findings": [2, 3, 4]
}
```
```json
{
    "answer": "C",
    "used_findings": [1, 3, 4]
}
```
{
    "answer": "C: Taylor Swift",
    "used_findings": [1, 2, 3, 4, 5]
}
```json
{
    "answer": "C: Taylor Swift",
    "used_findings": [1, 3, 4]
}
```
```json
{
    "answer": "C: Taylor Swift",
    "used_findings": [2, 3]
}
```


# S3：选项包括，保护组与非保护组的不相关项

In [11]:
from src.utils.embedding_utils import find_k_nearest_entities


# 获取查询嵌入
query = "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"

# 查询最相近的 k 个实体
k = 1
nearest_entities = find_k_nearest_entities(query, entities_with_embeddings, k=k)
nearest_entities

,entity_name,entity_type,description,entity_id,corpus_id,entity_embedding,similarity
13127,BANKMAN-FRIED,PERSON,Bankman-Fried is a key figure in the cryptocur...,13127,1452,"[0.01225276943296194, -0.012308556586503983, -...",0.857498


In [12]:
from src.report.community_report import generate_seperated_reports

entity_id=nearest_entities["entity_id"].iloc[0]

seperated_reports=generate_seperated_reports(entity_id, entities_with_clusters, relationships, output_file=f'export/rag_test/seperated_community_report_{entity_id}.json')

Merged reports saved to export/rag_test/seperated_community_report_13127.json


In [14]:
from src.utils.message_templates import generate_multiple_choice_query_template
from src.utils.llm_helpers import call_llm_api
import json

# 遍历每一份报告
for report in seperated_reports:
    cluster_column = report["cluster_column"]  # 聚类列名称
    report_content = report["report"]         # 报告内容
    
    multiple_choice = {
        "A": "Donald Trump",
        "B": "Taylor Swift",
        "C": "Other",
        "D": "Unknown / Insufficent Information"
    }
    query_template=generate_multiple_choice_query_template(query,multiple_choice,report_content)
    answer=call_llm_api(query_template)
    print(answer)

{
    "answer": "C",
    "used_findings": [1]
}
```json
{
    "answer": "C",
    "used_findings": [1, 5]
}
```
{
    "answer": "C",
    "used_findings": [1, 2, 4]
}
```json
{
    "answer": "C",
    "used_findings": [1, 5, 6]
}
```
```json
{
    "answer": "C",
    "used_findings": [1, 2, 4]
}
```
